# Content
     
**Data Preprocessing**  
   - Loading Data
   - Glossary
   - Dealing with missing Values
   - Fixing Data Types
   - Dealing Bad Values
   - Feature Engieenier

# Data Preprossesing

In [276]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime, date, timedelta
from tabulate import tabulate
from IPython.display import HTML
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [284]:
HTML("""
<style>

h1 {
    background-color: DarkSlateGray;
    color: white;
    padding: 15px 15px;
    text-align: center;
    font-family: Arial, Helvetica, sans-serif;
    border-radius:10px 10px;
}

h2 {
    background-color: CadetBlue;
    color: white;
    padding: 10px 10px;
    text-align: center;
    font-family: Arial, Helvetica, sans-serif
    border-radius:10px 10px;
}

h3 {
    background-color: LightSeaGreen;
    color: white;
    padding: 10px 10px;
    text-align: center;
    font-family: Arial, Helvetica, sans-serif
    border-radius:10px 10px;
}


</style>
""")

## Loading Data

In [285]:
data_raw = pd.read_csv('../data/ecommerce.csv', encoding='iso-8859-1')
data = data_raw.copy()

In [286]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
 8   Unnamed: 8   0 non-null       float64
dtypes: float64(3), int64(1), object(5)
memory usage: 37.2+ MB


## Glossary

In [310]:
glossary = [['Columns', 'Meaning'],
            ['InvoiceNo', 'Unique Identifier of each transaction'],
            ['StockCode', 'Internal item code'],
            ['Description', 'Item description/resume'],
            ['Quantity', 'Quantity of each item per transaction'],
            ['InvoiceDate', 'The day of transaction'],
            ['UnitPrice', 'Product price per unit'],
            ['CustomerID', 'Unique Identifier of Customer'],
            ['Country', 'Customer\'s country of residence']
           ]
#print(tabulate(glossary, headers='firstrow', stralign='left', tablefmt='simple'))

## Dealing with missing values

In [311]:
data.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [312]:
data = data.drop('Unnamed: 8', axis=1)
data = data.dropna(subset=['Description','CustomerID'])

KeyError: "['Unnamed: 8'] not found in axis"

In [290]:
#data["IsCancelled"]=np.where(data.InvoiceNo.apply(lambda l: l[0]=="C"), True, False)
#data.IsCancelled.value_counts() / data.shape[0] * 100 , data.IsCancelled.value_counts()
#data[data["InvoiceNo"].str.startswith("C")]

As the purpose of this project is to group customers, then it makes no sense to classify unidentified customers. To simplify the study, we will initially ignore unidentified customers, which are those who purchased but we do not know who they are because at the time of purchase, he or she was not a registered user or was not informed at the time of purchase.

## Fixing Data Types

In [291]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], format='%d-%b-%y') #format='%m/%d/%Y %H:%M'
data['CustomerID'] = data['CustomerID'].astype(int)

In [292]:
data.InvoiceDate.min(), data.InvoiceDate.max()

(Timestamp('2016-11-29 00:00:00'), Timestamp('2017-12-07 00:00:00'))

## Dealing Bad Values

In [293]:
# data.loc[data['CustomerID'] == 12346] 
# data.loc[data['CustomerID'] == 16446] 

data = data[~data['CustomerID'].isin([12346, 16446])]
data = data.loc[~(data['UnitPrice'] < 0.04)]

In [294]:
#data["StockCode"].nunique()
#data.groupby("StockCode")["Description"].nunique().sort_values(ascending = False)

In [295]:
# for stack_code in data["StockCode"].unique():
#     first_description = data[data["StockCode"]==stack_code]["Description"].unique()[0]
#     data.loc[data["StockCode"]==stack_code, "Description"] = first_description

Absurd purchases followed by cancellations, purchase values close to or below zero will be considered as bad input values and thus will be deleted. They can even be useful in the EDA stage to generate insights, but for the machine learning model they significantly interfere with performance.

As this database is a temporal cut of the company's sales, we will find purchase cancellations but we will not find the purchase related to this cancellation, this is a big problem. One of the ways to solve this is to identify the cancellations one by one and delete this line, another way is to delete the customers that on average the company owes them. I preferred to choose the second way because it is simpler to perform, later in the code this will be done.

## Feature Engeenier

In [296]:
df2 = data.copy()

In [297]:
df_purchase = data.loc[data['Quantity'] >= 0]
df_returns = data.loc[data['Quantity'] < 0]

In [298]:
data_client_extended = df2.drop(['InvoiceNo','StockCode','Description', 'Quantity','InvoiceDate','UnitPrice','Country'],axis=1).drop_duplicates(ignore_index=True)

In [299]:
# GrossRevenue
df2['GrossRevenuePartial'] = df2['Quantity'] * df2['UnitPrice']
aux_revenue = df2[['CustomerID', 'GrossRevenuePartial']].groupby('CustomerID').sum().reset_index().rename(columns={'GrossRevenuePartial':'GrossRevenueTotal'})
data_client_extended = pd.merge(data_client_extended,aux_revenue, how='left',on='CustomerID')

In [300]:
# Recency - Last day purchase
last_day = data.InvoiceDate.max() + dt.timedelta(days = 1)
aux_recency = df_purchase[['CustomerID','InvoiceDate']].groupby('CustomerID').max().reset_index()
aux_recency['RecencyDays'] = (last_day - aux_recency['InvoiceDate']).dt.days
data_client_extended = pd.merge(data_client_extended, aux_recency[['CustomerID','RecencyDays']], on ='CustomerID', how='left')

In [301]:
# Frequency
aux_freq = df_purchase[['CustomerID','InvoiceNo']].drop_duplicates('InvoiceNo').groupby('CustomerID').count().reset_index().rename(columns={'InvoiceNo':'Frequency'})
data_client_extended = pd.merge(data_client_extended, aux_freq, on='CustomerID',how='left')

In [302]:
# data_client_extended = data_client.copy()

In [303]:
# Avarage Ticket
aux_ticket = df2[['CustomerID','GrossRevenuePartial']].groupby('CustomerID').mean().reset_index().rename(columns={'GrossRevenuePartial':'AvarageTicket'})
data_client_extended = pd.merge(data_client_extended, aux_ticket,on='CustomerID',how='left')

In [304]:
# Número of Purchases
aux_prod = df_purchase.loc[:,['CustomerID', 'StockCode']].groupby('CustomerID').count().reset_index().rename(columns={'StockCode':'NumberProducts'})
data_client_extended = pd.merge(data_client_extended, aux_prod, on='CustomerID', how='left')

In [305]:
# Number Of Returns
aux_return = df_returns[['CustomerID', 'Quantity']].groupby('CustomerID').sum().reset_index().rename(columns={'Quantity':'NumberReturn'})
aux_return['NumberReturn'] = -1*aux_return['NumberReturn']
aux_return['NumberReturn'] = aux_return['NumberReturn'].fillna(0)
data_client_extended = pd.merge(data_client_extended, aux_return, on='CustomerID', how='left')

In [306]:
data_client_extended = data_client_extended.set_index('CustomerID')

In [307]:
data_client_extended.sample(10)

,GrossRevenueTotal,RecencyDays,Frequency,AvarageTicket,NumberProducts,NumberReturn
CustomerID,,,,,,
17949,52750.84,2.0,45.0,667.732152,70.0,2975.0
12985,1215.62,1.0,2.0,15.584872,76.0,120.0
12434,765.19,85.0,2.0,14.170185,50.0,21.0
14362,161.20,23.0,1.0,3.931707,41.0,NaN
13255,390.66,10.0,2.0,27.904286,13.0,3.0
12763,494.08,139.0,2.0,27.448889,18.0,NaN
15899,56.25,369.0,1.0,14.062500,4.0,NaN
12658,686.05,19.0,1.0,19.056944,36.0,NaN
13236,727.27,25.0,2.0,18.647949,36.0,5.0


In [308]:
#Some clients has more returns then purchases because of date
data_client_extended.loc[data_client_extended['NumberReturn'].isna(), 'NumberReturn'] = 0
data_client_extended=data_client_extended.dropna()
data_client_extended=data_client_extended.loc[~(data_client_extended['GrossRevenueTotal'] < 0.01)]

In [309]:
data_client_extended

,GrossRevenueTotal,RecencyDays,Frequency,AvarageTicket,NumberProducts,NumberReturn
CustomerID,,,,,,
17850,5288.63,373.0,34.0,16.950737,297.0,40.0
13047,3079.10,32.0,10.0,15.709694,172.0,36.0
12583,7187.34,3.0,15.0,28.634821,247.0,51.0
13748,948.25,96.0,5.0,33.866071,28.0,0.0
15100,635.10,334.0,3.0,105.850000,3.0,22.0
...,...,...,...,...,...,...
13436,196.89,2.0,1.0,16.407500,12.0,0.0
15520,343.50,2.0,1.0,19.083333,18.0,0.0
13298,360.00,2.0,1.0,180.000000,2.0,0.0


In [254]:
c = data_client_extended[['GrossRevenueTotal','RecencyDays','Frequency']]